# Practical 9 - Big Data Analytics (Part I)

Objective:  Understand some basic concepts and techniques for classification by predictive modelling, using a jupyter notebook. Understand linear and kernel SVMs and identify their strengths and weaknesses in application. Understand the importance of parameter selection and describe the cross validation technique for parameter selection. 

For this practical, your task is to review this notebook which outlines two examples of predictive modeling being applied to two datasets. All code blocks can be executed if there is no instruction asking for your code. You do need to read the notebook carefully and test everything by yourself.  Make sure you understand everything before continuing with the questions at the end and the final coding task.  

*N.B. all code blocks have to be excuted in sequence. If you encounter any difficulties, run the code from the very first block. In the final coding task, __DO NOT__ assume any preloaded packages or data set other than those appear in coding task blocks!*

## Iris Classfication

In this exercise, you will work through the Iris classification example discussed in the lecture, from data loading, classifier training and prediction. 

The **iris dataset** is an example dataset included in the `scikit-learn` package. But here we use `pandas` package to read it in from a `.csv` file to familiarise yourself with I/O. The **iris dataset** is widely used to demonstrate predictive modelling (classification) techniques in machine learning. The data set consists of 50 samples from each of three species of the Iris flower genus (_Iris setosa_, _Iris virginica_ and _Iris versicolor_). Four features were measured from each sample (in centimetres):
1. Sepal length
2. Sepal width
3. Petal length
4. Petal width

This results in a dataset with 150 records (50 samples \* 3 species of iris) where each record contains 4 measurements. The code below imports some libraries we will use for the data analysis and plotting. A brief description of each library is given below.
- `sklearn` is a library for machine learning. It also comes with the iris dataset.
- `numpy` is a library for numerical calculations.
- `pandas` is a library for manipulating data such as tables.
- `seaborn` is a statistics and plotting library.
- `mlxtend` is a library containing some additional tools for machine learning.

_N.B. if you need to install any of the packages, you should be able to use conda to get them into anaconda. But try a simple import first_  

In [ ]:
%matplotlib inline
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
from mlxtend.evaluate import plot_decision_regions

In [ ]:
iris = pd.read_csv('iris.csv')  # load the iris dataset
iris.head(5)

The purpose of this dataset is to find a model that can predict which species of iris (_setosa, versicolour, virginica_) a flower belongs to given these measurements. We call the species we are attempting to classify _targets_. For ease of use in a programming environment, each target corresponds to a numerical label value:

|target | species |
|-----------------|
|0 | setosa |
|1 | versicolour |
|2 | virginica |

To start, we need to create training and testing sets from the raw dataset. This can be done with the following statement that splits the initial dataset into 60% (90 examples) and 40% (60 examples) subsets for training and testing respectively.

In [ ]:
training, testing = train_test_split(iris, train_size=0.6)

This produces four arrays that we have named `training_data`, `testing_data`, `training_labels` and `testing_labels`. As an example, `training_data` contains 90 samples from the original dataset (i.e. 90 records of the 4 physical iris measurements) and `training_labels` is an array of 90 integers representing which species the elements of `training_data` belong to (e.g. the species of iris that the first record in `training_data` belongs to is given as the first record in `training_labels`). Remember that the species have been given an integer identifier rather than storing the names as strings.

We now want to examine the statistical properties of training examples from different targets (species) in terms of mean values and standard deviations for each feature. This will help us identify which features differ the most between species.

### Mean values by target

In [ ]:
iris.groupby('target').mean()  # the groupby methods forces the mean to be calculated per target

### Standard deviation by target


In [ ]:
iris.groupby('target').std()

### Group mean standard deviation

In [ ]:
iris.groupby('target').mean().std() # Group mean stdandard deviations

It seems that the two features `petal_length` and `petal_width` differ greatly between species. Their group means vary more than other two features, indicated by large group mean std devs shown above. The larger the group mean, the further apart they are. 

The discriminative nature of features can be verified with correlation analysis between feature values and target values. Calculating the correlation coefficient between the target (the target is just an arbitrary integer) and the feature measurements tells us how effective that feature is at distinguishing species. If the coefficient is close to -1 or 1, then the two are strongly correlated, if the value is close to 0, then the values are very weakly correlated.

In [ ]:
for feature in iris.columns:
    # print the correlation coefficient between the feature and the species (target)
    if feature != 'target':
        print 'Correlation between {} and target: '.format(feature) + str(iris[feature].corr(iris['target'])) + '\n'

From the results above, we can see that features `petal_length` and `petal_width` are the best candidate features for iris classification, confirming our general observation of the mean statistics earlier. This can be visualised by plotting the measurements of all samples against each other as the following.

In [ ]:
sns.pairplot(iris, hue='target', vars=('sepal_length', 'sepal_width', 'petal_length', 'petal_width'))

The off-diagonal plots are pairwise scatter plots, e.g. the left bottom corner is the scatter plot of `petal_width` against `sepal_length`. The diagonal plots are histograms of the property. All of these plots are grouped according to species, i.e. each color for each species. The above plots again show that `petal_length` and `petal_width` are the most strongly correlated features against the species of iris. Notice how the points for each species in the plots containing these features form distinct groups, in contrast to `sepal_length` vs `sepal_width` where the species is not clearly distinguishable especially between _versicolour_ and _virginica_ (green and red).

Another way to think about this is to imagine the plot colours were removed (i.e. you did not know the species of each point) and you were to pick one point at random. How confident would you be in determining the species of iris in each of those plots?

### Linear SVM

We will now train a linear SVM classifier on the training set and compute the accuracy on training and test datasets respectively using the `petal_length` and `petal_width` features. It is important to set up the predictive model (i.e. the classifier) on the training set and test its performance on the test set. The purpose of predictive modelling is to create models that are able to make predictions based on future data. Hence it is important to keep training and test data separate and not to use test data for training predictive models.

In [ ]:
clf = LinearSVC()

# Train the linear SVC using the training dataset
clf.fit(training[['petal_length','petal_width']], training['target'])

# Test the linear SVC accuracy using the testing dataset
print('Test accuracy =', clf.score(testing[['petal_length','petal_width']], testing['target'])*100, '%')

We demonstrate the classification results visually by plotting the decision boundaries produced by the linear SVM in the following code. Different colored regions correspond to different classes. 

In [ ]:
ax = plot_decision_regions(training[['petal_length','petal_width']].values, training['target'].values, clf=clf, legend=2)
ax.set_title('Linear SVM')
ax.set_xlabel('Petal Length')
ax.set_ylabel('Petal Width')

A linear SVM can also be trained by using all input features. In many cases, this will produce better test results than using a subset of features. 

In [ ]:
clf = LinearSVC()
clf.fit(training[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], training['target'])
print('Test accuracy =', clf.score(testing[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], testing['target'])*100, '%')

As can be seen, we achieve better test accuracy with linear SVM trained on all features compared to the previous result using only feature 2 and feature 3. Note that only test performances are important for prediction modelling. (Why?) 

## Two-Moons Dataset 

SVM classifiers can also be used to deal with nonlinear data. In the following example, we will show this using a dataset in which the two classes of data form interlocking moon shapes. First run the following block of code to generate and plot the dataset. 

In [ ]:
training_moon = pd.read_csv('moon_training.csv')
testing_moon = pd.read_csv('moon_testing.csv')
print training_moon[:3]

In [ ]:
sns.lmplot(x='A', y='B', data=training_moon, hue='target', fit_reg=False)

This is a binary classification problem, where points from each class form a moon shaped cloud opening in opposite direction, as shown in different colours in the above figure. Obviously, points from two classes can not be separated with a linear classifier. Next we train and test this problem with both linear SVM and kernel SVM.

In [ ]:
# give an easier to read name to the datasets
training_data = training_moon[['A','B']]
training_target = training_moon['target']
testing_data = testing_moon[['A','B']]
testing_target = testing_moon['target']


# train and test linear svm classifier
clf_linear = LinearSVC()
clf_linear.fit(training_data, training_target)
clf_linear_accuracy = clf_linear.score(testing_data, testing_target)*100
print('Accuracy of Linear SVM =', clf_linear_accuracy, '%')

# train and test kernel svm classifier
clf_kernel = SVC()
clf_kernel.fit(training_data, training_target)
clf_kernel_accuracy = clf_kernel.score(testing_data, testing_target)*100
print('Accuracy of Kernel SVM =', clf_kernel_accuracy, '%')

As expected, the kernel SVM achieves better performance (higher accuracy) than the linear SVM for this example. Next, we plot the decision boundary of each classifier model. 

In [ ]:
ax = plot_decision_regions(training_data.values, training_target.values.astype(int), clf=clf_linear, legend=2)
ax.set_title('Linear SVM')
ax.set_xlabel('A')
ax.set_ylabel('B')

In [ ]:
ax = plot_decision_regions(training_data.values, training_target.values.astype(int), clf=clf_kernel, legend=2)
ax.set_title('Kernel SVM')
ax.set_xlabel('A')
ax.set_ylabel('B')

As you can see from the above demonstration, the nonlinear kernel SVM produces a nonlinear decision boundary (i.e. a curve) to separate points from two classes, where regions that belong to different classes are shown in different colors. In contrast, linear SVM produces a linear decision boundary (i.e. a line) to separate points from two classes, which is not good enough for this example. 

Despite its effectiveness, kernel SVM is slower than linear SVM in training. We demonstrate this by training both linear and kernel SVM classifiers 3 \* 100 times and measuring the average time it takes to train each model.

In [ ]:
clf_linear = LinearSVC()
print('Linear SVM: ')
%timeit -n 100 -r 3 clf_linear.fit(training_data, training_target)

clf_kernel = SVC()
print('Kernel SVM: ')
%timeit -n 100 -r 3 clf_kernel.fit(training_data, training_target)

So far, we have used the default parameters for training and testing the kernel SVM classifier. The performance can be further improved with parameter tuning, specifically the regularisation parameter C plays an important role in final performance. We show how training and testing accuracies are affected by kernel SVM models trained with different C values.

In [ ]:
clist = 2**np.array(range(-2, 10), dtype='float')
traccuList = []     # list of training accuracies
tsaccuList = []     # list of testing accuracies
# train and test SVM for each c value in clist
for c in clist:
    # train a kernel SVM with C parameter value equal to c
    clf = SVC(C=c)
    clf.fit(training_data, training_target)
    # calculate the training and testing accuracies with given c value
    # and add them to the corresponding lists
    traccuList.append(clf.score(training_data, training_target)*100)
    tsaccuList.append(clf.score(testing_data, testing_target)*100)

In [ ]:
# plot the training accuracy values for different values of parameter C
# C values are in log-scale, i.e. -1 for 0.5, 0 for 1, i for 2**i
sns.plt.plot(np.log2(clist), traccuList, 'o-', label='Training Accuracy')

# plot the testing accuracy against C values
sns.plt.plot(np.log2(clist), tsaccuList, 'o-', label='Testing Accuracy')

# change some plot parameters and display result
sns.plt.xticks(np.log2(clist), [str(x) for x in clist])
sns.plt.legend(loc='upper left')
sns.plt.show()

Both training and testing performances are affected by the choice of parameter C. Increasing the value of C leads to an improvement in training performance, but not necessarily in testing performance because of overfitting.
In practice, we can't tune the parameters based on testing-set accuracy and have to do this on the training dataset. (Why?) To solve this problem, we need to use cross-validation, an effective approach for parameter selection on the training dataset. 

Next, we use k-fold cross-validation (`k=3`) to test the cross-validation accuracy for different C values and pick the best C that achieves the highest accuracy for cross-validation. We then retrain the kernel SVM classifier on the whole training dataset using the selected C value and test it on the testing dataset. 

In [ ]:
from sklearn import cross_validation

clist = 2**np.array(range(-2, 10), dtype='float')
cvscores = []
for c in clist:
    clf = SVC(C=c)
    scores = cross_validation.cross_val_score(clf, training_data, training_target, cv=3)
    cvscores.append(scores.mean()*100)
bestscore, bestC = max([(val, clist[idx]) for (idx, val) in enumerate(cvscores)])
print('Best CV accuracy =', bestscore, 'achieved at C =', bestC)
# retrain on whole trainning set using best C value obtained from Cross validation
clf = SVC(C=bestC)
clf.fit(training_data, training_target)
accu = clf.score(testing_data, testing_target)*100
print('Test accuracy =', accu, 'achieved at C =', bestC)

The accuracy value achieved by the kernel SVM classifier trained with the optimal parameter is higher than that produced with the kernel SVM classifier trained using the default parameter value. This demonstrates the importance and effectiveness of parameter selection. 

# Questions

Answer the questions below using the examples and code provided above. Double-click this cell and then edit the text below after each question. 

1. How many features are there for the iris dataset? How many examples? How many labels? 

2. Why is it important to split the dataset into training and test sets? Why does a classification model need to be trained on the training set and why does the prediction performance need to be measured on the test set?

3. How does correlation analysis can help identify the best features for the classification task? What are the best features for the iris data based on correlation analysis results?

4. Which class is easier to identify than the other two classes for the iris dataset? How can you tell? 

5. Which classification model produces better test results for the iris data? Linear SVM trained on all features or linear SVM trained on the two best features? What does this tell you?

6. Why does linear SVM not produce a good result for the two-moons example? 

7. Compare linear and kernel SVM in terms of predictive performance and training speed. What conclusions can you make?

8. Why do we need to perform parameter selection in training classification models for predictive modelling? 

9. Why can't we choose the classifier parameter that produces the best training performance? 

10. What is cross validation and why it is an effective technique for parameter selection in classifier training? 

# Coding task: Construct a classifier to predict handwritten digits using images

We use the `sklearn` package's `digits` data set. The following helps you understand the data. 

In [ ]:
from sklearn.datasets import load_digits
import numpy as np
digits = load_digits()

print('The number of images is: '+ str(digits.data.shape[0]))
print('The number of pixels in an image is: '+ str(digits.data.shape[1]))
print('The targets/labels are: '+ str(np.unique(digits.target)))

import matplotlib.pyplot as plt 
plt.gray() 
plt.matshow(digits.images[100]) #Show the 101st image in the data set.
plt.show()

In [ ]:
# Show all 10 digits
for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.imshow(digits.images[i], cmap=plt.cm.gray, vmax=16, interpolation='nearest')
    plt.xticks(())
    plt.yticks(())
plt.show()


The data is in `digits.data` and the labels are in `digits.target`. Images have been "squashed" to vectors and stored in `digits.data`. Construct a classifier to able to predict digits for a given new image. Please refer to the demonstration code blocks above if you have difficulties. 

### Your code goes here: 

In [ ]:
# Python code for digits classification
